# Amazon Video Games Review Analysis

In [116]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark-nlp") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.4.5") \
    .getOrCreate()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44921)
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:44921)

In [2]:
spark

In [62]:
sc = spark.sparkContext 
sc

<SparkContext master=yarn appName=spark-nlp>

In [114]:
from py4j.java_gateway import *
port = launch_gateway()
gateway = JavaGateway(
gateway_parameters=GatewayParameters(port=port),
callback_server_parameters=CallbackServerParameters(port=0))
random = gateway.jvm.java.util.Random()

In [115]:
sc.stop()
spark.stop()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44921)
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:44921)

In [111]:
data = spark.read.json("s3://qianyielva/amazon_game_data")

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44921)
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:44921)

In [103]:
data.show(10)

+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin|helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|B00002STAU| [0, 0]|    5.0|this is a old cla...|07 30, 2012|A1G0VFQ9198IUF|                  al|           a classic|    1343606400|
|B00002STAU| [2, 2]|    4.0|This game is more...|02 21, 2001| AXUOVXIGF9CKC|      "bigdcaldavis"|  good fighting game|     982713600|
|B00002STAU| [0, 0]|    5.0|If you love WWF n...|11 14, 2011|A15JTJXQXO22JJ|           Chad Frey|WWF Wrestlemania ...|    1321228800|
|B00002STAU| [1, 1]|    4.0|I had WWF Wrestle...|08 10, 2008| ANRNG7OAARR70|D. Hensley "Horro...|wrestling game wi...|    1218326400|
|B00002STAU| [0, 0]|    4.0|I have to admit I...|07 24, 2009|A

In [104]:
data = data.drop("helpful").drop("reviewerID").drop("unixReviewTime").drop("reviewTime").drop("reviewerName")

In [105]:
data.show(10)

+----------+-------+--------------------+--------------------+
|      asin|overall|          reviewText|             summary|
+----------+-------+--------------------+--------------------+
|B00002STAU|    5.0|this is a old cla...|           a classic|
|B00002STAU|    4.0|This game is more...|  good fighting game|
|B00002STAU|    5.0|If you love WWF n...|WWF Wrestlemania ...|
|B00002STAU|    4.0|I had WWF Wrestle...|wrestling game wi...|
|B00002STAU|    4.0|I have to admit I...|           A Classic|
|B00002STAU|    5.0|This game was ama...|The Best Wrestlin...|
|B00002STAU|    4.0|This right here i...|wrestling at it's...|
|B00002SVP7|    3.0|The Rampage Editi...|A few new levels ...|
|B00002SVP7|    2.0|Remember the mome...|                WTF?|
|B00002SVP7|    2.0|Back in 1993 Sega...|           Bo-oring!|
+----------+-------+--------------------+--------------------+
only showing top 10 rows



In [106]:
data.cache()

DataFrame[asin: string, overall: double, reviewText: string, summary: string]

### Clean Review Text

In [107]:
### make review text become lowercase
from pyspark.sql import functions as F
for col in data.columns:
    data = data.withColumn(col, F.lower(F.col(col)))
data.show(5)

+----------+-------+--------------------+--------------------+
|      asin|overall|          reviewText|             summary|
+----------+-------+--------------------+--------------------+
|b00002stau|    5.0|this is a old cla...|           a classic|
|b00002stau|    4.0|this game is more...|  good fighting game|
|b00002stau|    5.0|if you love wwf n...|wwf wrestlemania ...|
|b00002stau|    4.0|i had wwf wrestle...|wrestling game wi...|
|b00002stau|    4.0|i have to admit i...|           a classic|
+----------+-------+--------------------+--------------------+
only showing top 5 rows



In [110]:
data.show(10)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44921)
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:44921)

In [109]:
### remove punctuations and URL
data.createOrReplaceTempView("data")
Vasin = data.select('reviewText').rdd.flatMap(lambda x: x).collect()
def lower_clean_str(x):
    punc='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    lowercased_str = x.lower()
    for ch in punc:
        lowercased_str = lowercased_str.replace(ch, '')
    return lowercased_str

l=sc.parallelize(Vasin)
one_RDD = l.map(lower_clean_str)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44921)
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:44921)

In [100]:
data.reviewText = one_RDD.collect()

In [101]:
data.reviewText

['this is a old classic wrestling game  you can choose from shawn michals undertaker irs bret hart and a couple more  for it to say wwf and genesis both mean that it is a very old game',
 'this game is more of a oneonone fighter a la mortal kombat than a wrestler but both styles are incorporated into a good fighting game get this game instead of the snes version the genesis version has 8 characters but the snes has only six',
 'if you love wwf now its called wwe this is a great video game from sega genesisyou can pick 8 superstars any mode from easy to hard and what title belt to go through the gamethe royal rumble match is not easy to beat in hard mode you have to fight all 8 superstars in that levelwwf meets mortal kombat still a great game to own',
 'i had wwf wrestlemania the arcde game years ago when it first came out easy to play and hours of fun i had to buy it again because some douchebags stole my games at my house i bought it again after i bought the playstation game because 

In [51]:
### split the review text into words list
from pyspark.sql.functions import split
data = data.withColumn("reviewtext", split("reviewtext", "\s+"))
data.show(5)

+----------+-------+--------------------+--------------------+
|      asin|overall|          reviewtext|             summary|
+----------+-------+--------------------+--------------------+
|b00002stau|    5.0|[this, is, a, old...|           a classic|
|b00002stau|    4.0|[this, game, is, ...|  good fighting game|
|b00002stau|    5.0|[if, you, love, w...|wwf wrestlemania ...|
|b00002stau|    4.0|[i, had, wwf, wre...|wrestling game wi...|
|b00002stau|    4.0|[i, have, to, adm...|           a classic|
+----------+-------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
### remove 

In [47]:
data = data.toDF(*[c.lower() for c in data.columns])
data.show(5)

+----------+-------+--------------------+--------------------+
|      asin|overall|          reviewtext|             summary|
+----------+-------+--------------------+--------------------+
|B00002STAU|    5.0|this is a old cla...|           a classic|
|B00002STAU|    4.0|This game is more...|  good fighting game|
|B00002STAU|    5.0|If you love WWF n...|WWF Wrestlemania ...|
|B00002STAU|    4.0|I had WWF Wrestle...|wrestling game wi...|
|B00002STAU|    4.0|I have to admit I...|           A Classic|
+----------+-------+--------------------+--------------------+
only showing top 5 rows



In [24]:
import nltk
nltk.download('stopwords')
nltk.download('words')

[nltk_data] Downloading package stopwords to /home/hadoop/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /home/hadoop/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [25]:
# get the list of stopwords from nltk
from nltk.corpus import stopwords

eng_stopwords = stopwords.words('english')
eng_stopwords.append('xxxx')

#### Set up SparkNLP pipeline

In [26]:
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import (Tokenizer, Normalizer, 
                                LemmatizerModel, StopWordsCleaner)
from pyspark.ml import Pipeline

In [27]:
documentAssembler = DocumentAssembler() \
    .setInputCol('reviewText') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

In [28]:
# note normalizer defaults to changing all words to lowercase.
# Use .setLowercase(False) to maintain input case.
normalizer = Normalizer() \
    .setInputCols(['token']) \
    .setOutputCol('normalized') \
    .setLowercase(True)

In [29]:
# note that lemmatizer needs a dictionary. So I used the pre-trained
# model (note that it defaults to english)
lemmatizer = LemmatizerModel.pretrained() \
    .setInputCols(['normalized']) \
    .setOutputCol('lemma') \

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [30]:
stopwords_cleaner = StopWordsCleaner() \
    .setInputCols(['lemma']) \
    .setOutputCol('clean_lemma') \
    .setCaseSensitive(False) \
    .setStopWords(eng_stopwords)

In [31]:
# finisher converts tokens to human-readable output
finisher = Finisher() \
    .setInputCols(['clean_lemma']) \
    .setCleanAnnotations(False)

#### Defining the pipline

In [32]:
pipeline = Pipeline() \
    .setStages([
        documentAssembler,
        tokenizer,
        normalizer,
        lemmatizer,
        stopwords_cleaner,
        finisher
    ])

In [36]:
# transform text with the pipeline
equifax = pipeline.fit(data).transform(data.reviewText)

TypeError: Column is not iterable

In [34]:
equifax.show(10)

+----------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      asin|overall|          reviewText|             summary|            document|               token|          normalized|               lemma|         clean_lemma|finished_clean_lemma|
+----------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|B00002STAU|    5.0|this is a old cla...|           a classic|[[document, 0, 18...|[[token, 0, 3, th...|[[token, 0, 3, th...|[[token, 0, 3, th...|[[token, 10, 12, ...|[old, classic, wr...|
|B00002STAU|    4.0|This game is more...|  good fighting game|[[document, 0, 24...|[[token, 0, 3, Th...|[[token, 0, 3, th...|[[token, 0, 3, th...|[[token, 5, 8, ga...|[game, oneonone, ...|
|B00002STAU|    5.0|If you love WWF n...|WWF Wrestleman

### Import Modeling Packages 

In [9]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

### Feature Engineering 

In [17]:
stringIndexer_label = StringIndexer(inputCol="overall", outputCol="label")

In [18]:
stringIndexer_feature = StringIndexer(inputCol="reviewText", outputCol="features")

### Split dataset into training set and test set

In [19]:
splitted_data = data.randomSplit([0.8, 0.2])
train_data = splitted_data[0]
test_data = splitted_data[1]

print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))

Number of training records: 859944
Number of testing records : 215368


### Build LogisticRegression model(s) and train them using pipelines

In [20]:
logit = LogisticRegression(labelCol="label", featuresCol="features")

In [21]:
labelConverter = IndexToString(inputCol="prediction", 
                               outputCol="predictedLabel", 
                               labels=stringIndexer_label.fit(train_data).labels)

In [22]:
pipeline_logit = Pipeline(stages=[stringIndexer_label,stringIndexer_feature,logit,labelConverter])

### Train the model using training set

In [23]:
logit_model= pipeline_logit.fit(train_data)

IllegalArgumentException: 'requirement failed: Column features must be of type struct<type:tinyint,size:int,indices:array<int>,values:array<double>> but was actually double.'